# The HiddenMarkovModel distribution implements a (batch of) discrete hidden Markov models where the initial states, transition probabilities and observed states are all given by user-provided distributions.


---
# In this model, there is a sequence of integer-valued hidden states: z[0], z[1], ..., z[num_steps - 1] and a sequence of observed states: x[0], ..., x[num_steps - 1]. The distribution of z[0] is given by initial_distribution. The conditional probability of z[i + 1] given z[i] is described by the batch of distributions in transition_distribution. For a batch of hidden Markov models, the coordinates before the rightmost one of the transition_distribution batch correspond to indices into the hidden Markov model batch. The rightmost coordinate of the batch is used to select which distribution z[i + 1] is drawn from. The distributions corresponding to the probability of z[i + 1] conditional on z[i] == k is given by the elements of the batch whose rightmost coordinate is k. Similarly, the conditional distribution of x[i] given z[i] is given by the batch of observation_distribution. When the rightmost coordinate of observation_distribution is k it gives the conditional probabilities of x[i] given z[i] == k. The probability distribution associated with the HiddenMarkovModel distribution is the marginal distribution of x[0],...,x[num_steps - 1].


---

# A simple weather model.

* Represent a cold day with 0 and a hot day with 1
* Suppose the first day of a sequence has a 0.8 chance of being cold.
* We can model this using the categorical distribution:

```
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])

```

* Suppose a cold day has a 30% chance of being followed by a hot day
* and a hot day has a 20% chance of being followed by a cold day.

# We can model this as:

```
 transition_distribution = tfd.Categorical(probs=[[0.7, 0.3],
                                                 [0.2, 0.8]])

```
# Suppose additionally that on each day the temperature is
* normally distributed with mean and standard deviation 0 and 5 on
* a cold day and mean and standard deviation 15 and 10 on a hot day.
# We can model this with:



```
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])

```




In [14]:
import tensorflow as tf
import tensorflow_probability as tfp

## A simple weather model.

* Represent a cold day with 0 and a hot day with 1.
* Suppose the first day of a sequence has a 0.8 chance of being cold.
* We can model this using the categorical distribution:

In [15]:
tfd = tfp.distributions
initial_distribution = tfd.Categorical(probs=[0.8, 0.2])

* Suppose a cold day has a 30% chance of being followed by a hot day
* and a hot day has a 20% chance of being followed by a cold day.
* We can model this as:

In [16]:
transition_distribution = tfd.Categorical(probs=[[0.7, 0.3],
                                                 [0.2, 0.8]])

* Suppose additionally that on each day the temperature is
* normally distributed with mean and standard deviation 0 and 5 on
* a cold day and mean and standard deviation 15 and 10 on a hot day.
* We can model this with:

In [17]:
observation_distribution = tfd.Normal(loc=[0., 15.], scale=[5., 10.])

* We can combine these distributions into a single week long
* hidden Markov model with:

In [18]:
model = tfd.HiddenMarkovModel(
    initial_distribution=initial_distribution,
    transition_distribution=transition_distribution,
    observation_distribution=observation_distribution,
    num_steps=7)

# The expected temperatures for each day are given by:

In [19]:
mean=model.mean()  # shape [7], elements approach 9.0

In [20]:
with tf.compat.v1.Session() as sess:
  print(mean.numpy())

[3.        5.9999995 7.4999995 8.25      8.625001  8.812501  8.90625  ]


# The log pdf of a week of temperature 0 is:


In [21]:
model.log_prob(tf.zeros(shape=[7]))

<tf.Tensor: shape=(), dtype=float32, numpy=-19.855635>